# Predicting the Future of Product Sales and Crafting the Optimal Inventory Strategy




<h1>Introduction</h1>


<p>Navigating the ever-shifting landscape of customer demand is a core challenge that supply chain professionals face. Supply chain professionals often combine their expert intuition with some basic statistical techniques to infer the demand for products based on historical data. This process is relatively straightforward when dealing with smaller product categories and stable demand patterns. However, As businesses grow and encounter larger product categories with constantly shifting consumer demands, this traditional demand forecasting approach may not scale.
</p>

<p>
In this notebook, we will explore a more modern and scalable alternative – a data-driven, programmatic approach to demand forecasting. We will build a demand forecasting model with Python. We will also use this model for inventory optimization, covering concepts like reorder points, safety stock, and economic order quantity (EOQ).
</p>
<p>
Throughout this project, we will address two key business questions:
</p>
<ol>
<li>
<em>What is the demand forecast for the top-selling product in the next 24 months?</em>
</li>
<li>
<em>What is the optimal inventory level for the product?</em>
</li>
</ol>
<p>
Join me on this exploration as we seek answers, and feel free to share your thoughts and feedback on my approach.
</p>
<p>You can find the source code for this project at my <a href="https://github.com/Simontagbor/supply-chain-demand-forecast">github page.</a> You can also find the Jupyter notebook and the dataset on my <a href="https://www.kaggle.com/simontagbor/british-airways-predictive-ml">kaggle page</a>.</p>
</div>



<h1>Project Outline</h1>
<p>For this project, I completed the following tasks:</p>

- [x] Performed Exploratory Data Analysis.
- [x] Cleaned and Prepared the data for modeling.
- [x] Conducted Time Series Modeling With Prophet.
- [x] Evaluated the model performance.
- [x] Interpret the model results and answer the business questions.


## Problem Statement
<div class="card">
  <p>Large product categories and constantly shifting consumer demand patterns introduce a scaling challenge for traditional demand forecasting techniques. There is a need for an approach that reduces the level of guesswork and reduces the avoidable costly outcomes of poor inventory optimizations.</p>
</div>

## Prerequisites
<p> To follow along you need to have a basic understanding of the following:</p>
<ul>
<li>Python Programming. You can take a quick introduction to Python by following this <a href="https://www.pythonforbeginners.com/python-tutorial">tutorial</a></li>
<li>Time Series Analysis. You can take a quick <a href="https://zerotomastery.io/blog/time-series-forecasting-with-facebook-prophet/">introduction to Time Series Analysis</a> </li>
<li>Understanding of <a href="https://www.shipbob.com/inventory-management/inventory-optimization/">Inventory Optimization techniques.</a> will be a plus</li>
</ul>

<p>If you're already familiar with these concepts, you're good to go!</p>




<img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExaGpxN21laHVsY3Ixc2tsYmFvcGpmMjR1aHo4MXBsejY4azlncHpndCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/UxTBfyPE5JwRQbPCgU/giphy.gif" alt="gif" style="width: 40%; height: 100%;">	

## Project Dependencies


In [ ]:
#| code-fold: true
#| code-summary: "Show the code"
# import project libraries
import pandas as pd
import numpy as np # for linear algebra
import math # for math operations 

import seaborn as sns # for plotting

# handling files
import os 
import sys 

# data preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


# Model Building and Fitting
from sklearn.ensemble import RandomForestClassifier
from prophet import Prophet



# Model Evaluation and Tuning
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt # for plotting
import squarify # for tree maps

# Exploratory Data Analysis
<p>
The working <a href="https://www.kaggle.com/datasets/shashwatwork/dataco-smart-supply-chain-for-big-data-analysis">dataset</a> contains entries of customer demand information The data contains 53 features (columns)
</p>
<p>
To understand the data, let's perform some exploratory data analysis. We will  use the following techniques:
</p>
<ul class="task-list">
<li><input type="checkbox" checked="">Visual inspection of data.</li>
<li><input type="checkbox" checked="">Exploratory Data Visualizations. (Univariate and Bivariate)</li>
</ul>

## Visual Inspection of Data

In [ ]:
df = pd.read_csv("data/DataCoSupplyChainDataset.csv", encoding="ISO-8859-1")
df.head(2)

<p>To explore the spread of the data, we will use the describe() method to get the summary statistics of the data.</p>

In [ ]:
# retrieve the number of columns and rows
df.describe()

<details><summary>Click to see some Notable Observation of the Data</summary>
<ol type="1">
<li><p>Aproximately <code>55%</code> of orders had late delivery risks.</p></li>
<li><p>Aproximately <code>75%</code> of products cost <code>$199.99</code></p></li>
<li><p>All the products are available.</p></li>
<li><p>75% of customers bought goods worth at least <code>$247.40</code></p></li>
</ol>
<p>
Further inspection of the data will help us understand the data better.
</p>
</details>


## Data Preprocessing

<p>
we will focus on historical sales data, and product attributes like; stock level, and product category, we will also analyze the impact of other variables that contribute to demand patterns including geographic factors, customer segments and lead time.
</p>

### Preprocessing Tasks

- [x] Drop irrelevant columns
- [x] Drop rows with missing values
- [x] Create new features
- [x] Convert categorical features to numerical features

<p>Based on the above, we will drop the majority of the columns that are not relevant for forecasting the demand and extract new features from the existing columns
</p>



#### Drop Irrelevant Columns


In [ ]:
#| code-fold: true
#| code-summary: "Show the code"

# drop irrelevant columns
def drop_columns(df, columns_to_drop):
    try:
        df = df.drop(columns=columns_to_drop)
        print(f"{len(columns_to_drop)} columns dropped successfully. Number of columns remaining: {len(df.columns)}")
        return df
    except KeyError as e:
        print(f"""Column(s): {e} not found in dataframe.
              
            No columns dropped.
            Please Check that the column names are correct.""")
        return df

# Specify the columns to keep
colums_to_keep = ['Days for shipping (real)', 
                  'Days for shipment (scheduled)',
                  'Customer Country',
                  'Sales per customer',
                  'Delivery Status', 
                  'Late_delivery_risk', 
                  'Customer City',
                  'Customer Segment',
                  'Sales','Shipping Mode',
                  'Type', 'Product Card Id',
                  'Customer Zipcode', 
                  'Product Category Id', 
                  'Product Name',                    
                  'Product Price',
                  'Market', 
                  'Product Status',
                  'shipping date (DateOrders)',]

# Specify the columns to drop
columns_to_drop = [col for col in df.columns if col not in colums_to_keep ]

df = drop_columns(df, columns_to_drop)

#### Drop Rows with Missing Values

In [ ]:
# drop customer Zip code.
df = df.drop(columns=['Customer Zipcode'])

##### A Quick Spot Check for Missing Values

In [ ]:
### Check for Missing values
def check_null_values(df):
    null_values = df.isnull().sum()
    if null_values.sum() == 0:
        print("No null values found ✅")
    else:
        print("⚠️ Null values found in the following columns:")
        for column, null_count in null_values.iteritems():
            if null_count > 0:
                print(f"{column}: {null_count}")

# Use the function
check_null_values(df)

<p>In the code above, `df.isnull().sum()` returns a Series where the index is the column names and the values are the count of null values in each column. If the sum of these counts is 0, it means there are no null values in the DataFrame, so it prints "No null values found". Otherwise, it iterates over the Series and prints the column names and counts of null values.</p>

#### Create New Features
<p>The dataset contains a `shipping date` column which is a `DateTime` object from which we can extract `Month`, `Year`, `Day` and `Day of Week` that can be useful in our analysis.</p>

- [x] `Month` - to capture the months per sale.
- [x] `Year` - to capture the year per sales.
- [x] `Day` - to capture the day per sales.
- [x] `Day of Week` - to capture the day of the week per sales.

<p>
we need to also create a new  `Lead Time` column which is the difference between the `Days for shipment (scheduled)` and the `Days for shipping (real)`. This will help us understand the impact of lead time on demand.
</p>

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"

# Create month, Year, Day, and Weekday columns from Shipping Date
def extract_date_parts(df, date_column, prefix):
    try:
        df[date_column] = pd.to_datetime(df[date_column])
        df[f'{prefix} Year'] = df[date_column].dt.year
        df[f'{prefix} Month'] = df[date_column].dt.month
        df[f'{prefix} Day'] = df[date_column].dt.day
        df[f'{prefix} Weekday'] = df[date_column].dt.weekday
        # verify and notify that the columns have been created
        if f'{prefix} Year' in df.columns and f'{prefix} Month' in df.columns and f'{prefix} Day' in df.columns and f'{prefix} Weekday' in df.columns:
            print(f"✅ Success! Columns Created: {prefix} Year, {prefix} Month, {prefix} Day, and {prefix} Weekday")
            return df
        else:
            print("Error creating columns. Please check that the date column name is correct.")
    except Exception as e:
        print(f"Error creating columns: {e}")
        return df
# Add Lead Time Feature from Days for shipping (real) and Days for shipment (scheduled)
df['Lead Time'] = df['Days for shipping (real)'] - df['Days for shipment (scheduled)']

# Use the function to extract date parts
df = extract_date_parts(df, 'shipping date (DateOrders)', 'Shipping')


In [ ]:
# display the shape of the data frame
df.shape

<p>Now we have 23 columns and 180519 entries (rows) in the dataset.</p>

### Data Encoding
<p>The nature of categorical data makes it unsuitable for future analysis. For instance, machine learning models can't work with categorical values for customer origins like `UK`, `USA`, `France`, etc. We will convert these categorical values to numerical values using the `LabelEncoder` from the `sklearn` library.</p>

<p>I will also perform a <a href="https://www.geeksforgeeks.org/ml-one-hot-encoding-of-datasets-in-python/"> one-hot encoding</a> technique on categorical features for future machine learning modeling tasks.</p>

<p> I wrote a `prepare_data()` function that returns two preprocessed dataframes: one that is encoded using a label encoder function and the other encoded using one hot encoding technique.</p>

<p>You can learn about encoding techniques for categorical variables <a href="https://www.analyticsvidhya.com/blog/2020/08/types-of-categorical-data-encoding/">here</a></p>


In [ ]:
#| code-fold: true
#| code-summary: "Show the code"
# Select top selling product
top_product = df['Product Card Id'].value_counts().index[0]
# get top product ID
print(f"Filtering and Encoding Dataset for Top Product ID: {top_product}")

from sklearn.preprocessing import LabelEncoder

def prepare_data(df, product_card_id, categorical_cols, columns_to_drop):
    """
    Prepare a DataFrame for bivariate analysis and machine learnin
    g by applying label encoding and one-hot encoding to categorical 
    columns and dropping specified columns.

    Parameters:
    df (pandas.DataFrame): The original DataFrame.
    product_card_id (int): The product card ID to filter the DataFrame on.
    categorical_cols (list of str): The names of the categorical columns to apply encoding to.
    columns_to_drop (list of str): The names of the columns to drop from the DataFrame.

    Returns:
    pandas.DataFrame: The label encoded DataFrame for bivariate analysis.
    pandas.DataFrame: The one-hot encoded DataFrame for machine learning.
    """
    try:
        df_copy = df[df['Product Card Id'] == product_card_id].copy()  # create a copy

        # label encoding
        label_encoder = LabelEncoder()
        df_label_encoded = df_copy.copy()

        # Apply label encoding to categorical variables in place
        for col in categorical_cols:
            df_label_encoded[col] = label_encoder.fit_transform(df_label_encoded[col])

        # Drop specified columns
        df_label_encoded = df_label_encoded.drop(columns=columns_to_drop)

        # one-hot encoding
        df_one_hot_encoded = pd.get_dummies(df_copy, columns=categorical_cols)

        # Drop specified columns
        df_one_hot_encoded = df_one_hot_encoded.drop(columns=columns_to_drop)
        print("Data Encoding successful. ✅")
        return  df_one_hot_encoded, df_label_encoded
    except Exception as e:
        print(f"Error preparing data: {e}")
        return None, None

# Use the function to prepare the data for bivariate analysis
categorical_cols = ['Type', 'Customer Segment', 
                    'Delivery Status', 
                    'Customer City', 
                    'Market',
                    'Shipping Mode']

columns_to_drop = ['Product Name',
                   'Days for shipment (scheduled)', 
                   'Sales per customer', 
                   'Days for shipping (real)',
                   'Customer Country', 
                   'shipping date (DateOrders)', 
                   'Product Card Id', 
                   'Product Category Id', 
                   'Product Status', 
                   'Product Price']

# drop columns and encode data for correlation martrix and Machine learning
onehot_encode_df, label_encode_df = prepare_data(df, top_product, categorical_cols, columns_to_drop)

# rename Type column to Payment Type
label_encode_df = label_encode_df.rename(columns={'Type': 'Payment Type'})
onehot_encode_df = onehot_encode_df.rename(columns={'Type': 'Payment Type'})

#### Confirm Encoding of Dataset 

In [ ]:
label_encode_df.dtypes

In [ ]:
# validate the one-hot encoding
onehot_encode_df.dtypes

<p> The dataset is now ready for further analysis and modeling. we can now proceed to conduct exploratory data visualizations to understand the distribution of the data better.</p>

## Exploratory Data Visualizations
<p>To highlight the distributions of the individual variables as well as the relationship between the variables and the target variables, I used the following techniques:</p>

- [x] Univariate Analysis
- [x] Exploratory Time Series Analysis

### Univariate Analysis
<p>Univariate analysis is the simplest form of data analysis where the data being analyzed contains only one variable. Since it's a single variable, it doesn't deal with causes or relationships. The main purpose of univariate analysis is to describe the data and find patterns that exist within it.</p>

#### Visualizing the Distribution of the Dataset

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 15))
fig.suptitle('Distribution Plots for Selected Variables', 
             fontsize=16)
# Create a copy of the DataFrame
df_copy = df.copy()


# Plotting  the top ten products per Product Card Id
sns.countplot(data=df_copy, x='Product Card Id',
                color='blue', ax=axes[0, 0], 
                order=df_copy['Product Card Id'].value_counts().iloc[:10].index)
axes[0, 0].set_title('Distribution of Top Ten Product Id')
axes[0, 0].set_xlabel('Product Card Id')
axes[0, 0].set_ylabel('Count')


# Plotting Value of sales in  dollars
sns.histplot(data=df_copy, x='Sales', 
             kde=True, color='salmon', 
             bins=30, linewidth=2,
             ax=axes[0, 1])
axes[0, 1].set_title('Distribution of Sales')
axes[0, 1].set_xlabel('Sales value in Dollars')
axes[0, 1].set_ylabel('Frequency')


# Plotting Sales Value per customer
sns.histplot(data=df_copy, x='Sales per customer',
             bins=30, kde=True, linewidth=2,
             color='lightblue', ax=axes[0, 2])
axes[0, 2].set_title('Distribution of Sales per Customer')
axes[0, 2].set_xlabel('Sales per Customer')
axes[0, 2].set_ylabel('Frequency')

# Ploting the distribution of Product Price
sns.histplot(data=df_copy, x='Product Price', bins=30, kde=True, 
             color='lightgreen', linewidth=2, ax=axes[1, 0])

axes[1, 0].set_title('Distribution of Product Price')
axes[1, 0].set_xlabel('Product Price')

# ploting a tree map for Customer Segment
squarify.plot(sizes=df_copy['Customer Segment'].value_counts(), 
              label=df_copy['Customer Segment'].value_counts().index, 
              color=sns.color_palette("Set3"), ax=axes[1, 1])
axes[1, 1].set_title('Distribution of Customer Segment - Treemap')

# ploting a tree map for Top Ten Product Category Id
squarify.plot(sizes=df_copy['Product Category Id'].value_counts().iloc[:10],
                label=df_copy['Product Category Id'].value_counts().iloc[:10].index,
                color=sns.color_palette("Set2"), ax=axes[1, 2])
axes[1, 2].set_title('Distribution of Top Ten Product Category Id - Treemap')

# Plotting the distribution of Delivery Status
sns.countplot(data=df_copy, x='Delivery Status',
                color='pink', ax=axes[2, 0])
axes[2, 0].set_title('Distribution of Delivery Status')
axes[2, 0].set_xlabel('Delivery Status')
axes[2, 0].set_ylabel('Count')


# Plotting the distribution Payment Type with stacked bar chart
df_copy.groupby(['Type'])['Type'].count().plot(kind='bar', 
                                               stacked=True,
                                               ax=axes[2, 1])

axes[2, 1].set_title('Distribution of Payment Type')
axes[2, 1].set_xlabel('Payment Type')
axes[2, 1].set_ylabel('Count')

# Plotting the Distribution of top ten Customer Country
sns.countplot(data=df_copy, x='Customer Country',
                color='orange', ax=axes[2, 2], 
                order=df_copy['Customer Country'].value_counts().iloc[:10].index)
axes[2, 2].set_title('Distribution of Customer Country')
axes[2, 2].set_xlabel('Customer Country')
axes[2, 2].set_ylabel('Count')



# Adjust layout
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
# Show the plots
plt.show()


<details>
<summary>
Click to Read My Observations!
</summary>
<p>
The Top Selling <code>Product ID</code> is <code>365</code> which corresponds to a <code>product name</code>: <code>Perfect Fitness Perfect Rip Deck</code> this indicates a fast-moving product. I will focus the demand forecasting process on this product going forward
</p>
<p>
The distribution of <code>Sales Value</code> and <code>Sales per customer</code> are both positively skewed with a long tail. This indicates that the majority of sales are for low-value products. This is an interesting insight because it may suggest that the majority of customers are price-sensitive.
</p>
<p>
The distribution of <code>Product Price</code> is also positively skewed with a long tail. This means that the majority of products are low-value products.
</p>
<p>
The distribution of <code>Customer Segment</code> indicates that the majority of customers are from the consumer segment.
</p>
</details>



**NOTE**: Based on the insight from the univariate analysis, The rest of the analysis and forecasting will focus on the top-selling `Product Card Id` (365 ‘Perfect Fitness Perfect Rip Deck’)




## Exploratory Time Series Visualisation
<p>To understand the demand patterns of the top-selling product, let's create a time series heatmap to visualize the demand patterns of the top-selling product over time.</p>

### Time Series HeatMap of The Demand

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"	
import seaborn as sns
import matplotlib.pyplot as plt
import calendar

# Extract shipping date (DateOrders) and Sales columns
df_heatmap = df[['shipping date (DateOrders)', 'Sales']]
# Assuming 'df' is your original dataframe

df_heatmap.set_index('shipping date (DateOrders)', inplace=True)
resampled_df = df_heatmap.resample('M').sum()  # Resample to yearly frequency
# Set x-axis ticks to represent months and years
month_labels = [calendar.month_abbr[m.month] + '-' + str(m.year) for m in resampled_df.index]
# Plot the heatmap
plt.figure(figsize=(20, 10))
sns.heatmap(resampled_df.T, cmap='YlGnBu', cbar_kws={'label': 'Sales'})
plt.xticks(ticks=range(len(month_labels)), labels=month_labels, rotation=80, ha='right')

plt.title('Time Series Heatmap of Sales (Aggregated by Month)')
plt.xlabel('Month and Year')


plt.show()

<p>Judging from consistency in the shades of the heatmap, we can see that the demand for the top-selling product is fairly stable over time. However, it is interesting to note that the number of sales recorded for the first quarters of <code>2015</code>, <code>2016</code> and <code>2017</code> remained consistent however in 2018 the number of sales recorded in the first quarter dipped significantly. This is an interesting insight that we can explore further.
</p>
<p>
Next, Let's use the <code>Prophet</code> library to model the demand for the top-selling product. This will help us understand the cyclical patterns in the demand for the top-selling product
</p>



<h1>
Forecasting Demand with Prophet
</h1>
<p>
Prophet is a forecasting tool developed by Facebook. It is designed for analyzing time series data that display patterns on different time scales such as yearly, weekly, and daily. It also has advanced capabilities for modeling the effects of holidays on a time series and implementing custom seasonalities. see the documentation <a href="https://facebook.github.io/prophet/docs/quick_start.html">here</a>
</p>

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"

# import prophet
from prophet import Prophet

prophet_df = df.copy()
prophet_df = prophet_df.rename(columns={'shipping date (DateOrders)': 'ds', 'Sales': 'y'})
# Add custom Puerto Rico holidays
# Read the CSV file
holidays_df = pd.read_csv('data/puertorican_holidays.csv')

# Rename the 'Date' column to 'ds' and the 'Name' column to 'holiday'
holidays_df = holidays_df.rename(columns={'Date': 'ds', 'Name': 'holiday'})

# Drop the 'Type' column as it's not needed
holidays_df = holidays_df.drop(columns=['Type'])

# Add 'lower_window' and 'upper_window' columns
holidays_df['lower_window'] = 0
holidays_df['upper_window'] = 1

# Convert 'ds' to DateTime
holidays_df['ds'] = pd.to_datetime(holidays_df['ds'])

# Create a Prophet instance and provide the holidays DataFrame
prophet = Prophet(holidays=holidays_df)

prophet.fit(prophet_df)

# Create a DataFrame with future dates for forecasting
future = prophet.make_future_dataframe(periods=365, freq='D')

# Generate forecasts
forecast = prophet.predict(future) 

<p>
The code above uses the <code>Prophet</code> library to model the demand for the top-selling product. The model is trained on the <code>Sales</code> and <code>Shipping Date</code> columns. The model is then used to forecast the demand for the top-selling product over the next 365 days.
</p>
<p>
The code also included Puerto Rican holidays to account for the impact of holidays on the demand for the top-selling product. This is important because holidays can have a significant impact on demand patterns.
</p>
<p>
You might wonder why Puerto Rican holidays were included in the model. From the univariate analysis conducted earlier, we discovered that most of the orders were coming from Puerto Rico. The <code>forecast</code> variable now contains the forecasted values for the top-selling product. we will work with the variable later but for now, let’s evaluate the accuracy of our prophet model
</p>

<h3>Evaluating the Accuracy of the Time Series Forecast</h3>

<p>To determine the accuracy of the prophet model, we will use the `cross_validationa()` function provided by `Prophet`</p>

In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics
# Perform cross-validation
df_cv = cross_validation(model=prophet, initial='730 days', period='365 days', horizon='365 days')

<p>
The <code>cross_validation()</code> function performs cross-validation on the model. It trains the model on a subset of the data and then evaluates the model on the remaining data. This is a good way to evaluate the accuracy of the model. The <code>initial</code> parameter specifies the size of the training set. The <code>period</code> parameter specifies the frequency of the forecast.
</p>
<p>
Let’s visualize the performance of the model
</p>

In [ ]:
# Plot MAPE
from prophet.plot import plot_cross_validation_metric
#  set fig size
plt.figure(figsize=(9, 6))
fig = plot_cross_validation_metric(df_cv, metric='mape')
fig.get_axes()[0].get_lines()[0].set_markerfacecolor('#ADD8E7')  # Change color of the dots
fig.get_axes()[0].get_lines()[0].set_markeredgecolor('#ADD8E7')  # Change color of the dot edges

<p>
The forecast has lower `MAPE` (Mean Absolute Percentage Error) values for horizons within the 200-day range however the accuracy drops for horizons beyond 250 days. This suggests that the model is making more errors at periods beyond 250 days.
</p>
<p>
The model will be most useful to stakeholders if it can forecast demand beyond 250 days with a lower percentage of errors. Exposing the model to more historical data may help lower the MAPE significantly. nonetheless, let’s explore if there are opportunities to improve the accuracy by finding the best combinations of hyperparameters for the prophet model. I will use a <a href="https://aws.amazon.com/what-is/hyperparameter-tuning/#:~:text=Hyperparameter%20tuning%20allows%20data%20scientists,the%20model%20as%20a%20hyperparameter.">hyperparameter tuning</a> technique to try to optimize the model’s performance.
</p>
<h4 class="anchored">
Finding the Best Hyperparameter Combination for Lower MAPE
</h4>

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"
from sklearn.model_selection import ParameterGrid

# Assuming prophet_df is your DataFrame with 'ds' and 'y' columns
prophet_df = df.copy()
prophet_df = prophet_df.rename(columns={'shipping date (DateOrders)': 'ds', 'Sales': 'y'})

# Specify hyperparameter values to try
param_grid = {
    'seasonality_mode': ["additive", 'multiplicative'],
    'seasonality_prior_scale': [1, 5, 10, 20],
    'holidays_prior_scale': [5, 10, 20, 25],
    'changepoint_prior_scale': [0.005, 0.01, 0.05, 0.1]
}

# Generate all combinations of hyperparameters using ParameterGrid
param_combinations = ParameterGrid(param_grid)

<p>
The code above uses the <code>ParameterGrid</code> function from the <code>sklearn</code> library to create a grid of hyperparameters. The grid contains different combinations of hyperparameters for the prophet model.
</p><p>
On the other hand, the code below uses the <code>cross_validation()</code> function to evaluate the accuracy of the model for each combination of hyperparameters. The code then selects the combination of hyperparameters that results in the lowest MAPE.
</p>

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"
from itertools import product
# Store results in a dictionary
results = {}
print(f"trying all {len(param_combinations)} hyperparameter combinations")
# Generate all combinations of hyperparameters
param_combinations = list(product(*param_grid.values()))

for params in param_combinations:
    # Create a Prophet instance with current hyperparameter values
    prophet = Prophet(**dict(zip(param_grid.keys(), params)))

    # Fit the model
    prophet.fit(prophet_df)

    # Perform cross-validation
    df_cv = cross_validation(model=prophet, initial='730 days', period='365 days', horizon='365 days')


    # Calculate performance metrics
    df_metrics = performance_metrics(df_cv, rolling_window=0)

    # Store metrics in the results dictionary
    results[params] = df_metrics['mape'].mean()


:::{.callout .note}

**NOTE**: The code took a very long time to complete. It tried 128 different combinations of hyperparameters and the best model was the one with the lowest MAPE value.
::: 

<p>
The results are in! The best model had the following hyperparameters:
</p>

In [ ]:
# Find the hyperparameters with the lowest RMSE
best_hyperparams = min(results, key=results.get)
print(f"Best Hyperparameters: {dict(zip(param_grid.keys(), best_hyperparams))}")

<div class="cell-output cell-output-stdout">
<pre><code>Best Hyperparameters: {'seasonality_mode': 'additive', 'seasonality_prior_scale': 1, 'holidays_prior_scale': 5, 'changepoint_prior_scale': 0.005}</code></pre>
</div>

<p>Now let’s rebuild the model with the best hyperparameters and evaluate the model’s performance.</p>

In [ ]:
tuned_prophet = Prophet(holidays=holidays_df, 
                        seasonality_mode='additive', 
                        seasonality_prior_scale=1, 
                        holidays_prior_scale=5, 
                        changepoint_prior_scale=0.005)
# fit the model
tuned_prophet.fit(prophet_df)
# Create a DataFrame with future dates for forecasting
future = tuned_prophet.make_future_dataframe(periods=365, freq='D')

# Generate forecasts
new_forecast = tuned_prophet.predict(future)

<h5>Cross Validation of the Best Model</h5>

In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics
# Perform cross-validation
tuned_df_cv = cross_validation(model=tuned_prophet, initial='730 days', period='365 days', horizon='365 days')

<p>let’s compare the accuracy of the model before and after hyperparameter tuning.</p>

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"
fig, axs = plt.subplots(1, 2, figsize=(15, 9))

# Plot the first cross-validation metric
fig1 = plot_cross_validation_metric(df_cv, metric='mape', ax=axs[0])
fig1.get_axes()[0].get_lines()[0].set_markerfacecolor('#ADD8E7')  # Change color of the dots
fig1.get_axes()[0].get_lines()[0].set_markeredgecolor('#ADD8E7')  # Change color of the dot edges
# add title
axs[0].set_title('Initial Cross-Validation score MAPE')

# Plot the second cross-validation metric
fig2 = plot_cross_validation_metric(tuned_df_cv, metric='mape', ax=axs[1])
fig2.get_axes()[0].get_lines()[0].set_markerfacecolor('#ADD8E7')  # Change color of the dots
fig2.get_axes()[0].get_lines()[0].set_markeredgecolor('#ADD8E9')  # Change color of the dot edges
# add title
axs[1].set_title('Tuned Cross-Validation score MAPE')

plt.tight_layout()
plt.show()

<p>Not Exactly the outcome I was expecting but the tuned model’s performance remains consistent with the previous model. This may suggest that the model is not sensitive to the hyperparameters. Nonetheless, the model is still useful for forecasting demand for the top-selling product.</p>

<h4 class="anchored">
Forecast Results
</h4>
<p>As indicated earlier, the <code>forecast</code> variable contains the forecasted values of our <code>Sales</code> time series. Based on this forecast we will calculate the optimal inventory policy for this specific product.</p>
<p>
The <code>forecast</code> variable is a dataframe that contains the following columns:
</p>

In [ ]:
forecast.head(2)

<p>Before calculating the optimal inventory policy, let’s visualize the forecasted sales data. To have a feel for the seasonalities and cycles in the forecasted sales data</p>

<h4 class="anchored">
Visualizing Forecasted Sales
</h4>

In [ ]:
import warnings

# Ignore the specific FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)

# Plot the forecast
tuned_prophet.plot_components(new_forecast)

<h1 class="anchored">
 Business Question #1 
</h1>



<div style="text-align:center;">
<img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExeWJlamcxOTJnM3ZyeDllcXE4MW1lOHpmMndrMnJpMms0YnZ4aWNwcyZlcD12MV9naWZzX3NlYXJjaCZjdD1n/xT5LMWNOjGqJzUfyve/giphy.gif" alt="gif" style="width: 70%; height: 70%; display: block; margin: auto; right:20px;">
</div>



><em><h5>What is the demand forecast for the top selling product in the next 24 months?</h5></em>



<p>
<p>The sales trend between <code>2015</code> and <code>2017</code> marks a cycle where sales for the product remained relatively stable during the second and third quarters of each year and then dipped slightly in October with a sharp increase between November and December.
</p>
<p>The forecasted sales for the next 24 months, on the other hand, indicate a very stable demand pattern.
<p>
The zero variance observed in the product price may account for the relatively stable sales pattern forecasted for `2018` and `2019`. It might also be worth investigating the factors that may account for the cyclical dips between 2015 and 2017
</p>
<p>
We can also observe the impact of the Puerto Rican holidays on the forecasted sales.
</p>
<h1 class="anchored">
Finding Optimal Inventory Policy Based on Forecasted Demand
</h1>
<p>
Now that we have forecasted the demand for the top-selling product, we can use the forecasted demand to calculate the optimal inventory policy for the product.</p>
<p> 
Finding the optimal inventory policy will help us determine the <a href= "https://cashflowinventory.com/blog/reorder-point">Reorder Point</a>, <a href="https://cashflowinventory.com/blog/reorder-point">safety stock</a>, and <a href="https://cashflowinventory.com/blog/reorder-point" >Economic Order Quantity</a>(EOQ) for the product. These markers will help us ensure that we have the right amount of inventory on hand to meet customer demand while minimizing inventory costs.
</p>

<h2>Re Order Point</h2>
The reorder point is the inventory level at which we should reorder more stock. 
`ROP`  is calculated as the product of the average sales per day and the lead time (also referred to as `Lead Time Demand`) plus the `Safety stock`.

$$ Reorder\ Point = \text{Lead\ Time\ Demand} + \text{Safety\ Stock} $$


let's Find the Lead Time Demand

$$ \text{Lead Time Demand} = \text{Average Sales Per Day} \times \text{Lead Time} $$

:::{.callout .note}

<p> <strong>NOTE:</strong> It is important to state the difference between the downstream and upstream lead time.</p>
<p>The `Lead Time` with regards to the reordering point is the time it takes for a product to be delivered by upstream supply chain manufacturers to the store's warehouse. It is the time between the placement of an order by the store and the receipt of the product.</p>
<p> The current dataset only provides the `Days for shipment (scheduled)` and the `Days for shipping (real)` which only helps us determine downstream lead times. The downstream lead time is the time it takes for a product to be delivered to the customer after it has been ordered. This is not the lead time we need to calculate the reorder point.</p>

:::



<p>For the purposes of our analysis we will assume an average upstream `Lead time` of 7 days </p>

In [ ]:

# Extract average forecasted sales per day
average_forecasted_sales = new_forecast['yhat'].mean()


# Extract the average lead time
average_lead_time = 7  # 7 days
print(f"Average Lead Time: {average_lead_time}")

lead_time_demand = average_forecasted_sales * average_lead_time
print(f"Lead Time Demand: {lead_time_demand}")

<p>One final piece to the Reorder Point puzzle is the ` Safety Stock`. The safety stock is the extra stock that is kept on hand to mitigate the risk of stockouts due to uncertainties in demand and lead time.</p>

$$ \text{Safety Stock} = (\text{Maximum Daily Sales} \times \text{Maximum Lead Time}) - \text{Lead Time Demand}$$


<p>Let's also assume that there have been delays from the manufacturer in the past and the maximum lead time is 10 days. That is Three days later than the average order fulfillment timeline</p>

In [ ]:
# find maximum daily forecasted sales
max_daily_forecasted_sales = new_forecast['yhat'].max()
print(f"Maximum Daily Forecasted Sales: {max_daily_forecasted_sales}")

# find maximum lead time
max_lead_time = average_lead_time + 3  # 3 days delays in delivery than the average
print(f"Maximum Lead Time: {max_lead_time}")

# calculate safety stock
safety_stock = (max_daily_forecasted_sales * max_lead_time) - lead_time_demand
print(f"Safety Stock: {safety_stock}")

<p>Finally, we can calculate the reorder point for the top-selling product.</p>

<h3>Putting It All Together</h3>

In [ ]:
# calculate reorder point
reorder_point = lead_time_demand + safety_stock
print(f"The Optimal Reorder Point for the Top-selling Product is: {reorder_point}")

<p> As indicated by the result, the reorder point for the top-selling product is `3753` units, which means that we should reorder more stock when the inventory level reaches `3753` units. This will help us ensure that we have enough stock on hand to meet customer demand while minimizing inventory costs.</p>

<h3>Economic Order Quantity (EOQ)</h3>
<p> Alternatively, we can use the <a href="https://dclcorp.com/blog/inventory/economic-order-quantity-eoq/">Economic Order Quantity (EOQ) model</a> to calculate the optimal order quantity for the top-selling product. The EOQ model helps us determine the optimal order quantity that minimizes the total inventory costs.</p>

<p>Unlike the Reorder Point which is concerned with determining the level of inventory at which a new order should be placed to avoid stockouts, EOQ takes into account the costs of ordering (e.g., setup costs) and the costs, of holding inventory (e.g., storage costs, opportunity costs).</p>



<p>The Economic Order Quantity (EOQ) formula is given by:</p>

$$ EOQ = \sqrt{\frac{2DS}{H}} $$

<p>Where:</p>

- \( D \) is the demand rate (number of units demanded),
- \( S \) is the ordering cost per order,
- \( H \) is the holding cost per unit per year.

<p>This formula helps in determining the optimal order quantity that minimizes the total inventory costs.</p>


:::{.callout .note}
**NOTE**: We can figure out the demand rate \( D \) based on the existing data. However, the ordering cost \( S \)  and holding cost \( H \)  are not provided in the dataset. For our analysis, We will assume that the ordering and holding cost is `10%` and `30%` of the product price respectively.

:::


<h4>Estimating Holding Cost and Ordering Cost</h4>
<h5>Holding Cost</h5>
<p>Holding costs typically include expenses related to storing inventory, such as warehousing, insurance, and security. It also includes the opportunity cost of tying up capital in inventory.</p> 

<h5>Ordering Cost</h5>
<p>Ordering costs are the expenses incurred in the process of ordering and receiving inventory. This includes the cost of preparing and processing purchase orders, receiving and inspecting the goods, and storing and managing the inventory.</p>

In [ ]:
# extract the product price of top selling product(product card id:365)
product_price = df[df['Product Card Id'] == 365]['Product Price'].iloc[0]
print(f"The Product Price is: {product_price}")
# calculate holding cost
H = 0.10 * product_price
# calculate ordering cost
S = 0.30 * product_price

# calculate forecasted demand rate
D = new_forecast['yhat'].mean()

print(f"The Demand Rate is: {D}")
print(f"The Holding Cost is: {H}")
print(f"The Ordering Cost is: {S}")

<h4>Putting It All Together</h4>

In [ ]:
EOQ = math.sqrt((2 * D * S) / H)
print(f"The Economic Order Quantity is: {EOQ}")

<p>Based on the EOQ model, the optimal order quantity for the top-selling product is `35` units. This means that we should order `35` units of the product at a time to minimize the total inventory costs. This will help us ensure that we have the right amount of inventory on hand to meet customer demand while minimizing inventory costs.</p>

<h1 class="anchored">
 Business Question #2 
</h1>



<div style="text-align:center;">
<img src="https://media.giphy.com/media/l4FBeaUCLhxrXPlpC/giphy.gif" alt="gif" style="width: 90%; height: 90%; display: block; margin: auto;">
</div>

See Angry Jack(from spongebob) being angry about his Massive inventory buildup. soon we will see how he could use our new inventory optimisations.

><em><h5>What is the optimal inventory level for the top selling product?</h5></em>

<div class="card">
<p>The optimal inventory policy for the top-selling product is as follows:</p>

- **Reorder Point**: 3753 units
- **Economic Order Quantity (EOQ)**: 35 units
- **Safety Stock**: 2284 units

<p>Here's what Angry Jack should do: 
</p>
<p>When the stock level of the top-selling product hits 3753 units, He needs to place an order of 35 units with his suppliers.</p>
<p>Inventory management decisions based on these markers will help the company ensure that there is the right amount of inventory of the top-selling product on hand to meet customer demand while minimizing inventory costs.</p>
</div>

<h1>Bonus: Investigating the Top Predictors of Demand Outcomes</h1>
<p>Now that we have forecasted the demand for the top-selling product and calculated the optimal inventory policy, we can investigate the top predictors of demand outcomes. This will help us understand the factors that contribute to demand patterns and identify opportunities to improve sales outcomes.</p>

<p>The top-selling product price has remained fixed throughout the dataset. We can safely rule out the product price as a predictor of demand outcome for the period under review.</p>

<h3>Wait....I have a Hypothesis</h3>
<p>I have a hunch that `Product Lead Time`, `Customer Segment` and some `Geographic factors` will be top predictors of demand outcomes.</p>
<p>I will test this hypothesis by conducting a feature importance analysis using the <a href="https://www.analyticsvidhya.com/blog/2021/06/understanding-random-forest/">Random Forest</a> algorithm.</p>

<h5><em>Let's Investigate.....</em></h5>

<div style="text-align:center;">
<img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExaTUzaDNxOW54cDU5MXl0emd3dzI4dnRqNjJpMWxpYW1sOGVrb2xjOSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/dgEtXsuqq2PL2/giphy.gif" alt="gif" style="width: 60%; height: 90%; display: block; margin: auto;">
</div>

<h4>Correlation Analysis</h4>
<p>Before we proceed with the feature importance analysis, let’s conduct a correlation analysis to identify the top predictors of demand outcomes.</p>

<p>
we will create the <a href="https://builtin.com/data-science/correlation-matrix#:~:text=A%20correlation%20matrix%20is%20a,1%20a%20not%20strong%20relationship.">correlation matrix</a> using the `corr()` method and then use the `heatmap()` function from the `seaborn` library to visualize the correlation matrix.</p>
<p>We will use the `label_encode_df` data frame created during the data preprocessing stage.</p>



In [ ]:
# correlation analysis
correlation_matrix = label_encode_df.corr()

# plot the correlation matrix
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', cbar_kws={'label': 'Correlation Coefficient'})
plt.title('Correlation Matrix of Selected Variables')
plt.show()

<h5>Observation</h5>
<p>The Heatmap shows there are no strong correlations between the features and the target variable. This suggests that the demand for the top-selling product is not strongly influenced by any single feature.</p>

<h4>Feature Importance Analysis Using Random Forest Regressor Model</h4>
<p>The correlation matrix could not identify the top predictors of demand outcomes. We will use the Random Forest algorithm to identify the top predictors of demand outcomes. The Random Forest algorithm is an ensemble learning method that uses multiple decision trees to make predictions. It is a powerful algorithm for feature importance analysis.</p>
<p> We will train a Random Forest Regressor model on the `onehot_encode_df` data frame and then use the `feature_importances_` attribute of the model to identify the top predictors of demand outcomes.</p>

<h5>Split the Data for Model Training</h5>

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"

from sklearn.model_selection import train_test_split

# prepare features excluding the sale
X_features = onehot_encode_df.drop(columns=['Sales'])
#  Drop Shipping year
X_features = X_features.drop(columns=['Shipping Year'])
# prepare target variable
y_target = onehot_encode_df['Sales']

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state=42)


<h5>Train the Random Forest Regressor Model</h5>

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"

from sklearn.ensemble import RandomForestRegressor

# create a random forest regressor model
rf_model = RandomForestRegressor(n_estimators=150, max_depth=10,  min_samples_split=2)

# fit rf odel to the training data
rf_model.fit(X_train, y_train)

<h3>Feature Importance Analysis</h3>

In [ ]:
#| code-fold: true
#| code-summary: "Show the code"
import matplotlib.cm as cm
# retrieve the feature importances
feature_importances = rf_model.feature_importances_

# get the top 5 feature importances
top_5_feature_importances = feature_importances.argsort()[-5:]

# visualize the top 5 features of importance
plt.figure(figsize=(10, 6))
colors = cm.viridis(np.linspace(0, 1, 
                                len(top_5_feature_importances)))
plt.barh(X_train.columns[top_5_feature_importances], 
         feature_importances[top_5_feature_importances],
          color=colors)
plt.xlabel('Feature Importance')

plt.title('Top 5 Feature Importances')
plt.show()


<h4>Discussion of Results</h4>

<p>The feature importance analysis confirms the majority of my hypothesis</p>
<p>We can see that Product `Lead Time`, and some `Geographic factors` were among the top predictors of demand outcomes</p>
<p>`Customer Segment` however was not among the top predictors of demand outcomes. This brings us to the end of our analysis</p>



# **Congratulations on making it This Far!**



<div style="text-align:center;">
<img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExb3V2N25lNWFrNGtwNGo5aTMxaWM2Yno0dXRmOHhtM2FvbnE0eW41eiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9cw/XEIa9GtOXdMN4KOaPn/giphy.gif" alt="gif" style="width: 100%; height: 1o0%; display: block; margin: auto;">
</div>

<h1>Key Takeaways</h1>
<p>If you made it this far, I hope you have enjoyed the journey. Here are the key takeaways from this project:</p>

- [x] We explored how to build a demand forecasting model with Python.
- [x] We used the model for inventory optimization, covering concepts like reorder points, safety stock, and economic order quantity (EOQ).
- [x] We discovered trends, seasonalities and holiday effects on the top-selling product based on the dataset provided.
- [x] We found the optimal inventory policy for the top-selling product.

<h4>How does this method offer enhanced scalability compared to conventional demand forecasting techniques?</h4>

<ol><li>The existing data processing steps can easily be adapted to accommodate larger product categories.
<li>Improved speed and accuracy.</li>
<li>Minimized guesswork and enhanced reproducibility for validating forecasts.</li>
</ol>
<p>Feel free to share your thoughts and feedback on my approach. I am open to learning and improving my skills.</p>


<h1>Up Next:</h1>


<div style="display:flex; gap:50px;">
<a href="https://simontagbor.github.io/BritishAirways-data-science/" style="text-decoration: none; color: black;">
<div style="width: 300px; border-radius: 15px; overflow: hidden; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); transition: transform 0.3s;">
  <img src="https://images.unsplash.com/photo-1615558568469-de10704d9102?q=80&w=1374&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D" alt="Blog Post Image" style="width: 100%; height: auto; border-radius: 15px 15px 0 0;">
  <div style="padding: 15px;">
  <div style="font-size: 1.2em; font-weight: bold; margin-bottom: 10px;">Understanding How Customers Feel About British Airways Flight Experience.</div>
  <div style="font-size: 0.9em; color: #555;">A Topic Modelling and Sentiment Analysis Approach</div>
  </div>
</div>
</a>
<a href="#" style="text-decoration: none; color: black;">
<div style="width: 300px; border-radius: 15px; overflow: hidden; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); transition: transform 0.3s;">
  <img src="https://images.unsplash.com/photo-1614028674026-a65e31bfd27c?q=80&w=1470&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D" alt="Blog Post Image" style="width: 100%; height: auto; border-radius: 15px 15px 0 0;">
  <div style="padding: 15px;">
  <div style="font-size: 1.2em; font-weight: bold; margin-bottom: 10px;">Supply Chain Demand Forecasting Using Prohet</div>
  <div style="font-size: 0.9em; color: #555;">Using  Prophet Algorithm to Implement a Demand Forecasting Model with Python(Currently A Work In Progress)</div>
  </div>
</div>
</a>
<a href="#"  style="text-decoration: none; color: black;">
<div style="width: 300px; border-radius: 15px; margin-right:15px; overflow: hidden; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); transition: transform 0.3s;">
  <img src="https://images.unsplash.com/photo-1616401784845-180882ba9ba8?q=80&w=1470&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D" alt="Blog Post Image" style="width: 100%; height: auto; border-radius: 15px 15px 0 0;">
  <div style="padding: 15px;">
  <div style="font-size: 1.2em; font-weight: bold; margin-bottom: 10px;">Inventory Optimisation With ARIMA and SARIMA</div>
  <div style="font-size: 0.9em; color: #555;">Using Time Series Analysis to forecast the future demand of a given product.(Work in Progress)</div>
  </div>
</div>
</a>

</div>